In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import requests
import json
import random
import warnings
import time
warnings.filterwarnings("ignore")

# to prevent us from being blocked for the site while crawling

In [2]:
http_proxies = [
    "http://111.230.140.187:8010",
    "http://148.70.217.163:3128",
    "http://123.7.62.246:808",
    "http://45.119.208.134:80",
    "http://124.71.162.246:808",
    "http://223.111.183.114:8080",
    "http://85.26.146.169:80",
    "http://218.78.61.241:808",
    "http://95.216.222.80:80",
    "http://47.111.18.39:8088",
]
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.7113.93 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
]
user_agent = random.choice(user_agent_list)
header = {'User-Agent': user_agent}

http_proxy = random.choice(http_proxies)
proxy_dict ={"http": http_proxy}


In [3]:
car={"modle":[],"price":[],"kilomaters":[],"stick":[],"engine":[],"year":[],"hand":[]
    ,"last_owner":[],"curr_owner":[],"city":[],"car_id":[]}
price = []
modle = []
KM = []
stick = []
engine = []
year = []
hand = []
last_owner = []
curr_owner = []
city = []
car_id=[]

In [4]:
next_page=''
page=''
start_url='https://www.komo.co.il/code/cars/private.asp'

In [5]:
for num in range(1,56):# loop for all the manufacturers
    for years in range(2001,2021,2):#loop for years range
        time.sleep(0.5)
        next_page=''
        base_url=start_url+'?yazranNumH='+str(num)+'&fromYear='+str(years)+'&toYear='+str(years+1)
        #base url for each manufacturer and year
        count=0
        #changing usr agents
        user_agent = random.choice(user_agent_list)#
        header = {'User-Agent': user_agent}
        r = requests.get(base_url,headers=header,proxies=proxy_dict)
        soup = BeautifulSoup(r.text,'html.parser')
        try:
            (soup.find('font',style='font-size:46px;').text=='לא נמצאו תוצאות.')#empty page
            print('epmty_page')
            continue
        except:
            pass
        pages=0
        for tag in soup.find_all('a',class_='paging'):
            pages+=1 #number of pages in each manufacturer and year filter
        for curr_page in range(1,pages):#loop to run on all the pages found for manufacturer and year filter
            time.sleep(1.5)
            if (curr_page==1):#1st page is the base url for the manufacturer and year filter
                page = base_url
            else: 
                page = start_url+next_page#next page is start url + the string for tne next page in the link 
            r = requests.get(page,headers=header,proxies=proxy_dict)
            soup = BeautifulSoup(r.text,'html.parser')
            try:
                next_page=soup.find("a",attrs={"rel":"next"})["href"]#next page link
            except:
                if (count==1):
                    break
                else:
                    count=1
                    pass
                
            x=[]
            for tagx in soup.find_all('table', class_= 'tblModaa round6 shadowPres CardStyle tblModaaArrow'):
                x.append(tagx.attrs['id'])#id for each listing
            y=[]
            for i in range(0,len(x)):#remove the string and only stay with the listing id
                y.append(x[i].replace('modaaRow',''))
                car_id.append(x[i].replace('modaaRow',''))

            for j in y:#loop for all the listings found in the current page4
                time.sleep(0.5)
                user_agent = random.choice(user_agent_list)
                header = {'User-Agent': user_agent}
                proxy_dict ={"http": http_proxy}
                #basic url for each listin wit xxx in place of the id num
                url = 'https://www.komo.co.il/code/cars/details/?modaaNum=XXXXXXXXXXXXXXXXXXX&mikumAll=1&mikumIski=0&clickSource=1'
                #the url of each listing in the loop
                modaa = requests.get(url.replace('XXXXXXXXXXXXXXXXXXX',j),headers=header,proxies=proxy_dict)
                soup = BeautifulSoup(modaa.text,'html.parser')
                table =[]#store partial data
                for tag in soup.ul.find_all('strong'):
                    table.append(tag.text)
                    
                table2 =[]#store partial data
                for tag in soup.find_all('div',class_='firstInfo'):
                    table2.append(tag.text)
                #placing all the data needed from listin in the arrays    
                stick.append(table[0])#stick
                curr_owner.append(table[3])#currhand
                last_owner.append(table[4])#lasthand
                year.append(table2[0])#year
                KM.append(table2[1])#KM
                hand.append(table2[2])#hand
                engine.append(table2[3])#engine
                price.append(soup.find('span',class_='ModaaWDetailsValue').text)#price
                city.append(soup.find('div',class_='addresBottom').text)#area
                modle.append(soup.find('h1',class_='modaaWTitleNew').text)#modle
                table.clear()
                table2.clear()
                
                
            x.clear()
            y.clear()



epmty_page
epmty_page
epmty_page
epmty_page
epmty_page
2230947
2234559
2247068
2254306
2237862
2238972
2246722
2252216
2222279
2214577
2197942
2243328
2215858
2200285
2237008
2230182
2102928
2220450
2243053
2254317
2240818
2239089
2246085
2253184
2224203
2238182
2248570
2227698
2242181
2241996
2249369
2248721
2237057
2213960
2245797
2053184
2250043
2255075
2253703
2247320
2219066
2227161
2216285
2235376
2234208
2252816
2102162
2236878
2249260
2232685
2244716
2232985
2249966
2244167
2247027
2248203
2185522
2242980
2214526
2243167
2234890
2240721
2240441
2224910
2238894
2240156
2239502
2236783
2203128
2227553
2235237
2235815
2135708
2234418
2249629
2240320
2247475
2245356
2225179
2223161
2188226
2228650
2186874
2252469
2243453
2217264
2254776
2251975
2222490
2203983
2244862
2234931
2253457
2246317
2231736
2250618
2250628
2228384
2249290
2231145
2246274
2247567
2243789
2244074
2245899
2244041
2244633
2244179
2094261
2232657
2157483
2154889
2220080
2215546
2241962
2222545
2248215
2217827
2

creating the Data frame

In [6]:
car["modle"]+=modle
car["kilomaters"]+=KM
car["stick"]+=stick
car["engine"]+=engine
car["year"]+=year
car["hand"]+=hand
car["last_owner"]+=last_owner
car["curr_owner"]+=curr_owner
car["city"]+=city
car["price"]+=price
car["car_id"]+=car_id

In [7]:
df=pd.DataFrame.from_dict(car,orient='index')
df=df.transpose()

In [8]:
df2=df.dropna(how='any')
df2=df2.replace ('\n','',regex=True)
df2

,modle,price,kilomaters,stick,engine,year,hand,last_owner,curr_owner,city,car_id
0,"רכב פרטי, אאודי A4","40,000 ₪",220000,ידנית,1984,2005,4,פרטית,פרטית,אזור באר שבע והסביבה,2230947
1,"רכב פרטי, אאודי A4","20,900 ₪",131000,אוטומטית,1984,2006,2,פרטית,פרטית,אזור באר שבע והסביבה,2234559
2,"רכב פרטי, אאודי A6","21,999 ₪",200,אוטומטית,3123,2006,3,פרטית,פרטית,אזור באר שבע והסביבה,2247068
3,"רכב פרטי, אאודי A4","38,000 ₪",195000,אוטומטית,1984,2006,4,פרטית,פרטית,אזור פתח תקוה והסביבה,2254306
4,"רכב פרטי, אאודי A4","5,000 ₪",218073,אוטומטית,1781,2006,3,פרטית,פרטית,אזור רמלה - לוד,2237862
...,...,...,...,...,...,...,...,...,...,...,...
10408,"רכב פרטי, רנו מגאן","102,000 ₪",28000,אוטומטית,1332,2019,1,4/2022,פרטית,אזור עכו - נהריה,2252687
10409,"רכב פרטי, רנו מגאן","98,000 ₪",60000,אוטומטית,1332,2019,1,5/2022,פרטית,אזור חיפה וחוף הכרמל,2252117
10410,"רכב פרטי, רנו מגאן","94,000 ₪",40000,אוטומטית,1332,2019,1,לא,פרטית,אזור ירושלים והסביבה,2137748
10411,"רכב פרטי, רנו מגאן",לא צוין מחיר,35000,אוטומטית,1461,2019,2,פרטית,פרטית,אזור יישובי השומרון,2254705


making sure we dont have duplicates form the uniqe listing id in car_id

In [11]:
df2.to_csv(r'D:\CARS.csv', index = False)